## Interval Analysis

In [1]:
# !pip install tensorboardX

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import time
import matplotlib.pyplot as plt

from torchvision import datasets, transforms
# from tensorboardX import SummaryWriter

use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")
batch_size = 64

np.random.seed(42)
torch.manual_seed(42)


## Dataloaders
train_dataset = datasets.MNIST('mnist_data/', train=True, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))
test_dataset = datasets.MNIST('mnist_data/', train=False, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Simple NN. You can change this if you want. If you change it, mention the architectural details in your report.
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc = nn.Linear(28*28, 200)
        self.fc2 = nn.Linear(200,10)

    def forward(self, x):
        x = x.view((-1, 28*28))
        x = F.relu(self.fc(x))
        x = self.fc2(x)
        # x = F.softmax(x, dim=-1) # added softmax for probabilities
        return x

class Normalize(nn.Module):
    def forward(self, x):
        return (x - 0.1307)/0.3081

# Add the data normalization as a first "layer" to the network
# this allows us to search for adverserial examples to the real image, rather than
# to the normalized image
model = nn.Sequential(Normalize(), Net())

model = model.to(device)
model.train()


Sequential(
  (0): Normalize()
  (1): Net(
    (fc): Linear(in_features=784, out_features=200, bias=True)
    (fc2): Linear(in_features=200, out_features=10, bias=True)
  )
)

In [2]:
from tqdm import tqdm

def train_model(model, num_epochs):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    for epoch in tqdm(range(num_epochs)):
        model.train()
        running_loss = 0.0
        # tbar = tqdm(total=len(train_loader), desc="items of one epoch:")
        for i, data in enumerate(train_loader, 0):
            images, labels = data
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        #     tbar.update(1)
        # tbar.close()

        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.3f}')

def test_model(model):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        # tbar = tqdm(total=len(test_loader), desc="items of test:")
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        #     tbar.update(1)
        # tbar.close()
        print(f'Accuracy on images: {100 * correct / total}')

In [3]:
train_model(model, 15)
test_model(model)

  7%|▋         | 1/15 [00:04<01:09,  4.99s/it]

Epoch 1/15, Loss: 0.593


 13%|█▎        | 2/15 [00:10<01:06,  5.11s/it]

Epoch 2/15, Loss: 0.297


 20%|██        | 3/15 [00:15<01:02,  5.20s/it]

...

 93%|█████████▎| 14/15 [01:13<00:05,  5.31s/it]

Epoch 14/15, Loss: 0.093


100%|██████████| 15/15 [01:18<00:00,  5.25s/it]

Epoch 15/15, Loss: 0.088


Accuracy on images: 97.06


### Write the interval analysis for the simple model

In [4]:
## TODO: Write the interval analysis for the simple model
## you can use https://github.com/Zinoex/bound_propagation

In [9]:
eps_list = [i/100 for i in list(range(1, 11))]
verified_cnt_dict = dict.fromkeys(eps_list, 0)
print(eps_list) # [0.01, ..., 0.10]

def ibp_forward_normalize(lower, upper):    
    lower = (lower - 0.1307) / 0.3081
    upper = (upper - 0.1307) / 0.3081
    return lower, upper

def ibp_forward_relu(lower, upper):
    # relu is like max(0,x)
    lower = torch.clamp(lower, min=0)
    upper = torch.clamp(upper, min=0)
    return lower, upper

def ibp_forward_linear(model, lower, upper):
    # https://github.com/Zinoex/bound_propagation/blob/65bd7f80fd4133b16a0525db1ae008a7354bff8d/src/bound_propagation/linear.py#L69
    center = (lower + upper) / 2
    width = upper - lower
    diff = width / 2
    weight = model.weight
    bias = model.bias
    # https://github.com/Zinoex/bound_propagation/blob/65bd7f80fd4133b16a0525db1ae008a7354bff8d/src/bound_propagation/linear.py#L29
    center, diff = center.unsqueeze(-2), diff.unsqueeze(-2)
    weight = weight.transpose(-1, -2)

    w_mid = center.matmul(weight)
    if bias is not None:
        w_mid = w_mid + bias.unsqueeze(-2)
    w_diff = diff.matmul(weight.abs())

    lower = w_mid - w_diff
    lower = lower.squeeze(-2)

    upper = w_mid + w_diff
    upper = upper.squeeze(-2)

    return lower, upper

def ibp_forward_softmax(lower, upper):
    lower_exp, upper_exp = torch.exp(lower), torch.exp(upper)
    sum_lower_exp, sum_upper_exp = lower_exp.sum(dim=-1, keepdim=True) , upper_exp.sum(dim=-1, keepdim=True)
    lower_softmax = lower_exp / sum_upper_exp
    upper_softmax = upper_exp / sum_lower_exp
    return lower_softmax, upper_softmax

def ibp(model, lower, upper):
    # We have 3 types of layers in the model here!
    
    # Linear
    # https://github.com/Zinoex/bound_propagation/blob/65bd7f80fd4133b16a0525db1ae008a7354bff8d/src/bound_propagation/linear.py#L67

    # Activation
    # https://github.com/Zinoex/bound_propagation/blob/65bd7f80fd4133b16a0525db1ae008a7354bff8d/src/bound_propagation/activation.py#L120

    # Normalization
    
    for layer in model:
        if isinstance(layer, Normalize): # as its the first layer
            lower, upper = ibp_forward_normalize(lower, upper)
        elif isinstance(layer, Net):
            # propagate through each layer now
            lower, upper = ibp_forward_linear(layer.fc, lower, upper)
            lower, upper = ibp_forward_relu(lower, upper)
            lower, upper = ibp_forward_linear(layer.fc2, lower, upper)
            # lower, upper = ibp_forward_softmax(lower, upper)
    return lower, upper

for data in test_loader:
    images, labels = data
    images, labels = images.to(device), labels.to(device)
    x = images.view(images.size(0), -1) # 64, 784
    for eps in eps_list:
        #hyper rectangle for the batch of data
        #https://github.com/Zinoex/bound_propagation/blob/65bd7f80fd4133b16a0525db1ae008a7354bff8d/src/bound_propagation/bounds.py#L160
        # lower = x - eps
        lower = torch.clamp(x-eps, 0.0, 1.0)
        # upper = x + eps
        upper = torch.clamp(x+eps, 0.0, 1.0)
        lower_final, upper_final = ibp(model, lower, upper)
        batch_size = x.size(0)
        for i in range(batch_size):
            label = labels[i].item()
            flag = True
            for j in range(10): # 10 classes 0 - 9 
                if lower_final[i, label] <= upper_final[i, j] and j != label:
                    flag = False; break
            if flag:
                verified_cnt_dict[eps] += 1

print("results:")
for eps in eps_list:
    acc = 100 * verified_cnt_dict[eps] / len(test_dataset)
    print(f"epsilon = {eps:.2f}, Accuracy: {acc:.2f}% = {verified_cnt_dict[eps]}/{len(test_dataset)}  ")

[0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]
results:
epsilon = 0.01, Accuracy: 31.93% = 3193/10000  
epsilon = 0.02, Accuracy: 1.09% = 109/10000  
epsilon = 0.03, Accuracy: 0.01% = 1/10000  
epsilon = 0.04, Accuracy: 0.00% = 0/10000  
epsilon = 0.05, Accuracy: 0.00% = 0/10000  
epsilon = 0.06, Accuracy: 0.00% = 0/10000  
epsilon = 0.07, Accuracy: 0.00% = 0/10000  
epsilon = 0.08, Accuracy: 0.00% = 0/10000  
epsilon = 0.09, Accuracy: 0.00% = 0/10000  
epsilon = 0.10, Accuracy: 0.00% = 0/10000  


10000